In [61]:
class Chromosome:
    '''
    Klasa predstavlja jedan hromozom za koji se cuva njegov genetski kod i vrednost funkcije prilagodjenosti.
    '''
    def __init__(self, content, fitness):
        self.content = content
        self.fitness = fitness
    
    def __str__(self):
        return "%s f=%d" % (self.content, self.fitness)


In [68]:
import random
import string

'''Klasa predstavlja implementaciju genetskog algoritma za resavanje problema prelaska nivoa.
    Koristi se:
    - Uniformno ukrstanje sa verovatnocom '_crossover_p'
    - Mutacija sa verovatnocom '_mutation_rate'
    - Turnirska selekcija sa parametrom '_tournament_k'
    - Zamena generacije se vrsi tako sto se od jedinki izabranih pri selekciji ukrstanjem
        pravi celokupna nova generacija (pogledati u knjizi alternativne zamene generacija)
'''
class GeneticAlgorithm:    
    def __init__(self, target):
        self._target = target                               # Niska koja se pogadja
        self._target_size = 10                              # Broj poteza koji je potrebno pronaci
        self._allowed_gene_values = [0, 1]                  # Dozvoljene vrednosti koje mogu biti u genu

        """Parametri genetskog algoritma, eksperimentalno izabrani."""
        self._iterations = 1000                             # Maksimalni dozvoljeni broj iteracija
        self._generation_size = 20                          # Broj jedinki u jednoj generaciji
        self._mutation_rate = 0.05                          # Verovatnoca da se desi mutacija
        self._reproduction_size = 1000                      # Broj jedinki koji ucestvuje u reprodukciji
        self._current_iteration = 0                         # Koristi se za interno pracenje iteracija algoritma
        self._crossover_p = 0.5                             # Verovatnoca za odabir bita prvog roditelja pri ukrstanju
        self._top_chromosome = None                         # Hromozom koji predstavlja resenje optimizacionog procesa
        self._tournament_k = 20                             # Parametar k za turnirsku selekciju
        
    def initial_population(self):
        """
        Generisemo _generation_size nasumicnih jedinki kao instance klase Chromosome i vracamo ih u listi.
        """
        init_pop = []                   # inicijalna populacija jedinki
        # ----------------------------------------------------------
        # STUDENTSKI DEO
        # ----------------------------------------------------------
        """Generisemo _generation_size nasumicnih jedinki."""
        allowed_symbols = [0, 1]
        init_pop = []                   # inicijalna populacija jedinki
        for i in range(self._generation_size):
            genetic_code = []           # genetski kod koji cemo nasumicno izabrati
            for k in range(self._target_size):
                # Uzimamo nasumicni gen iz dozvoljenih vrednosti za gen
                genetic_code.append(self.take_random_element(self._allowed_gene_values))
            # U inicijalnu generaciju dodajemo novi genetski kod
            init_pop.append(genetic_code)

        # Transformisemo listu tako da od liste genetskih kodova postane lista hromozoma
        init_pop = [Chromosome(content, self.fitness(content)) for content in init_pop]
        # ----------------------------------------------------------
        return init_pop
        
    def selection(self, chromosomes):
        """
        Funkcija bira self._reproduction_size hromozoma koristeci turnirsku selekciju
        koji ce se na dalje koristiti za ukrstanje i smenu generacija.
        """
        selected_chromos = []
        selected_chromos = [self.selection_tournament_pick_one(chromosomes, self._tournament_k) for i in range(self._reproduction_size)]

        return selected_chromos


    def selection_tournament_pick_one(self, chromosomes, k):
        """
        Bira jednu jedinku koristeci turnirsku selekciju.
        Ne vrsi normalizaciju i sortiranje po funkciji prilagodjenosti usled performansi.
        Parametar k definise koliko jedinki iz populacije se izvlaci.
        """

        # Biramo k nasumicnih jedinki iz populacije i trazimo jedinku
        # koja ima najvecu funkciju prilagodjenosti
        # Ovo predstavlja jednu od varijanti turnirske selekcije.
        the_chosen_ones = []
        top_i = None
        for i in range(k):
            pick = random.randint(0, len(chromosomes)-1)
            the_chosen_ones.append(chromosomes[pick])
            if top_i == None or the_chosen_ones[i].fitness > the_chosen_ones[top_i].fitness:
                top_i = i
        return the_chosen_ones[top_i]


    def the_goal_function(self, chromosome):
        """
         Za slucaj da smo nasli optimalno resenje (u opstem slucaju ovo retko znamo)
         pamtimo gen kao optimalni i prekidamo algoritam (bice zadovoljen kriterijum zaustavljanja).
         Iako u praksi ne znamo da li je dobijeno resenje optimalno, cesto postoji odredjena
         'funkcija cilja' koja testira da li je dobijeno resenje dovoljno kvalitetno da se prihvati.
        """
        if chromosome == self._target:
            self._top_chromosome = chromosome

    def fitness(self, chromosome):
        """Koristi gen da pokrene trcanje kroz nivo i vraca ishod - fitness."""
        self.the_goal_function(chromosome)
        return self.make_a_run(chromosome)


    def take_random_element(self, fromHere):
        """ Funkcija vraca nasumicno odabran element iz kolekcije 'fromHere'. """
        i = random.randrange(0, len(fromHere))
        return fromHere[i]
        
    def mutation(self, chromosome):
        """
        Vrsi mutaciju nad hromozomom sa verovatnocom self._mutation_rate
        Vraca mutirani hromozom ili originalni hromozom ukoliko do mutacije nije doslo.
        """
        # ----------------------------------------------------------
        # STUDENTSKI DEO
        # ----------------------------------------------------------
        t = random.random()
        if t < self._mutation_rate:
            # dolazi do mutacije
            print("Mutation occured with probability: {}".format(self._mutation_rate))
            i = random.randint(0, len(chromosome) - 1)
            chromosome = list(chromosome)
            chromosome[i] = random.choice(self._allowed_gene_values)
            # chromosome = "".join(chromosome)
        return chromosome

        # ----------------------------------------------------------
        # pass
        
        
    def create_generation(self, chromosomes):
        generation = []
        generation_size = 0
        
        while generation_size < self._generation_size:
            # Proizvoljno se biraju 2 roditelja za ukrstanje
            [parent1, parent2] = random.sample(chromosomes, 2)
            
            # Dobijaju se 2 detata ukrstanjem
            child1_code, child2_code = self.crossover(parent1.content, parent2.content)
            
            # Vrsi se mutacija nad decom
            child1_code = self.mutation(child1_code)
            
            # Prave se novi hromozomi
            child1 = Chromosome(child1_code, self.fitness(child1_code))
            child2 = Chromosome(child2_code, self.fitness(child2_code))
            
            # Dodaju se u generaciju
            generation.append(child1)
            generation.append(child2)
            
            generation_size += 2
            
        return generation
    
    def crossover(self, a, b):
        """
        Vrsi uniformno ukrstanje po verovatnoci self._crossover_p i vraca 2 nove jedinke.
        """
        # ----------------------------------------------------------
        # STUDENTSKI DEO
        # ----------------------------------------------------------
        ab = list(a)
        ba = list(b)
        for i in range(len(a)):
            p = random.random()
            if p < self._crossover_p:
                ab[i] = a[i]
                ba[i] = b[i]
            else:
                ab[i] = b[i]
                ba[i] = a[i]
        return (ab, ba)
        # ----------------------------------------------------------
        # pass
        
    def optimize(self):
        # Generisi pocetnu populaciju jedinki i izracunaj
        # prilagodjenost svake jedinke u populaciji
        chromosomes = self.initial_population()

        # Sve dok uslov zaustavljanja nije zadovoljen
        while not self.stop_condition():
            print("Iteration: %d" % self._current_iteration)

            # Izaberi iz populacije skup jedinki za reprodukciju
            for_reproduction = self.selection(chromosomes)

            # Prikaz korisniku trenutnog stanja algoritma
            the_sum = sum(chromosome.fitness for chromosome in chromosomes)
            print("Reproduction chromos sum fitness: %d" % the_sum)
            print("top solution: %s" % max(chromosomes, key=lambda chromo: chromo.fitness))

            # Primenom operatora ukrstanja i mutacije kreiraj nove jedinke
            # i izracunaj njihovu prilagodjenost.
            # Dobijene jedinke predstavljaju novu generaciju.
            chromosomes = self.create_generation(for_reproduction)
            self._current_iteration += 1
            print()

        # Vrati najkvalitetniju jedinku u poslednjoj populaciji
        if self._top_chromosome:
            return Chromosome(self._top_chromosome, self.fitness(self._top_chromosome))
        else:
            return max(chromosomes, key=lambda chromo: chromo.fitness)
        
    def stop_condition(self):
        return self._current_iteration > self._iterations or self._top_chromosome != None
    
    def make_a_run(self, chromosome, should_log=False):
        fitness = 0

        IDI_DESNO = 1
        SKOCI = 0
        RUPA = 0
        ZEMLJA = 1

        if len(chromosome) != len(self._target):
            sys.exit("Duzina hromozoma mora biti {}".format(len(self._target)))

        for i, potez in enumerate(chromosome):
            # Ako je igrac naleteo na rupu a nije skocio, zaustavljamo sa daljom
            # proverom i vracamo fitness koji je do sada izracunat.
            if self._target[i] == RUPA and potez == IDI_DESNO:
                if should_log: print("{}: Naleteo na rupu i nije skocio, kraj igre.".format(i))
                return fitness
            # Ako je naleteo na rupu i skocio, povecamo fitness
            elif self._target[i] == RUPA and potez == SKOCI:
                if should_log: print("{}: Preskocio rupu".format(i))
                fitness += 1
            # Ako je na zemlji i idi desno, povecamo fitness
            elif self._target[i] == ZEMLJA and potez == IDI_DESNO:
                if should_log: print("{}: Pomerio se udesno na zemlji".format(i))
                fitness += 1
            elif self._target[i] == ZEMLJA and potez == SKOCI:
                if should_log: print("{}: Skocio bez potrebe".format(i))
        if should_log: print("Presao nivo!")

        return fitness

    


In [69]:

genetic = GeneticAlgorithm([1, 1, 1, 0, 1, 0, 1, 0, 1, 1])

In [70]:
solution = genetic.optimize()
genetic.make_a_run(solution.content, True)

Iteration: 0
Reproduction chromos sum fitness: 67
top solution: [0, 1, 1, 0, 1, 0, 1, 0, 0, 0] f=7
Mutation occured with probability: 0.05
Mutation occured with probability: 0.05

Iteration: 1
Reproduction chromos sum fitness: 140
top solution: [0, 1, 1, 0, 1, 0, 1, 0, 1, 0] f=8
Mutation occured with probability: 0.05

Iteration: 2
Reproduction chromos sum fitness: 156
top solution: [1, 1, 1, 0, 1, 0, 1, 0, 1, 0] f=9

Iteration: 3
Reproduction chromos sum fitness: 172
top solution: [1, 1, 1, 0, 1, 0, 1, 0, 1, 0] f=9

Iteration: 4
Reproduction chromos sum fitness: 180
top solution: [1, 1, 1, 0, 1, 0, 1, 0, 1, 0] f=9

Iteration: 5
Reproduction chromos sum fitness: 180
top solution: [1, 1, 1, 0, 1, 0, 1, 0, 1, 0] f=9

Iteration: 6
Reproduction chromos sum fitness: 180
top solution: [1, 1, 1, 0, 1, 0, 1, 0, 1, 0] f=9

Iteration: 7
Reproduction chromos sum fitness: 180
top solution: [1, 1, 1, 0, 1, 0, 1, 0, 1, 0] f=9

Iteration: 8
Reproduction chromos sum fitness: 180
top solution: [1, 1, 1

10